In [1]:
from convertSolution import makeMap,convertSolution,cleanCSV

In [2]:
model_name = 'SolutionFile/basb-brrr-2024'
variables=['vgdp:EV']
csv_type='SES'

In [3]:
makeMap(model_name, variables)
convertSolution(model_name, csv_type)
cleanCSV(model_name, csv_type)